In [ ]:
# %% 01 08 / 27
# This code enables you to get the height values passing though coordinates durin 17 November 2021. 
# Input: processed_GEDI01_B_2021321104203_O16603_02_T09953_02_005_02_V002.h5, #
# VCS1764_PAA_UTM46N_2018.shp
# VCS1764_PAA_UTM46N_2018.geojson
# 

# %%
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
import geoviews as gv
from geoviews import opts, tile_sources as gvts
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

# %% [markdown]
# ## 1.2 Set Up the Working Environment and Retrieve Files<a id="1.2"></a>
# #### The input directory is defined as the current working directory. Note that you will need to have the jupyter notebook and example data (.h5 and .geojson) stored in this directory in order to execute the tutorial successfully.

# %%
inDir = os.getcwd() + os.sep  # Set input directory to the current working directory

L1B = "processed_GEDI01_B_2021321104203_O16603_02_T09953_02_005_02_V002.h5"
L1B

# %% [markdown]
# #### The standard format  for GEDI filenames is as follows:
# > **GEDI01_B**: Product Short Name    
# **2019170155833**: Julian Date and Time of Acquisition (YYYYDDDHHMMSS)  
# **O02932**: Orbit Number   
# **T02267**: Track Number  
# **02**: Positioning and Pointing Determination System (PPDS) type (00 is predict, 01 rapid, 02 and higher is final)  
# **003**: GOC SDS (software) release number  
# **01**: Granule Production Version  

# %% [markdown]
# #### Read in a GEDI HDF5 file using the `h5py` package.

# %%
os.getcwd()

# %%
L1B = "/Users/yayayapoop/Library/CloudStorage/OneDrive-TheUniversityofNottingham/Mres - Onedrive/15 04 23 - Sujit Earth engine shape file- WD/Working Folder/17 07 23 - Post Mid review working folder/17 07 23 - New Indices/27 07 23 - GEDI L2 Product most recent 59703/gedi-tutorials-main@d270c58295d/processed_GEDI01_B_2021333152446_O16792_03_T06042_02_005_02_V002.h5"

# %%
gediL1B = h5py.File(L1B, 'r')  # Read file using h5py

# %% [markdown]
# #### Navigate the HDF5 file below. 

# %%
list(gediL1B.keys())

# %% [markdown]
# #### The GEDI HDF5 file contains groups in which data and metadata are stored.
# #### First, the `METADATA` group contains the file-level metadata.

# %%
list(gediL1B['METADATA'])

# %% [markdown]
# This contains useful information such as the creation date, PGEVersion, and VersionID. Below, print the file-level metadata attributes.

# %%
for g in gediL1B['METADATA']['DatasetIdentification'].attrs: print(g) 

# %%
print(gediL1B['METADATA']['DatasetIdentification'].attrs['purpose'])

# %% [markdown]
# ## 2.2 Read SDS Metadata and Subset by Beam <a id="2.2"></a>

# %% [markdown]
# ####  The GEDI instrument consists of 3 lasers producing a total of 8 beam ground transects. The eight remaining groups contain data for each of the eight GEDI beam transects. For additional information, be sure to check out: https://gedi.umd.edu/instrument/specifications/.

# %%
beamNames = [g for g in gediL1B.keys() if g.startswith('BEAM')]
beamNames

# %% [markdown]
# #### One useful piece of metadata to retrieve from each beam transect is whether it is a full power beam or a coverage beam. 

# %%
for g in gediL1B['BEAM0000'].attrs: print(g)

# %%
for b in beamNames: 
    print(f"{b} is a {gediL1B[b].attrs['description']}")

# %% [markdown]
# #### Below, pick one of the full power beams that will be used to retrieve GEDI L1B waveforms in Section 3. 

# %%
beamNames = ['BEAM1011']

# %% [markdown]
# #### Identify all the objects in the GEDI HDF5 file below. 
# Note: This step may take a while to complete.

# %%
gediL1B_objs = []
gediL1B.visit(gediL1B_objs.append)                                           # Retrieve list of datasets
gediSDS = [o for o in gediL1B_objs if isinstance(gediL1B[o], h5py.Dataset)]  # Search for relevant SDS inside data file
[i for i in gediSDS if beamNames[0] in i][0:10]                              # Print the first 10 datasets for selected beam

# %% [markdown]
# ---
# # 3. Visualize a GEDI Orbit <a id="visualizeorbit"></a>
# #### In the section below, import GEDI L1B SDS layers into a `GeoPandas` GeoDataFrame for the beam specified above. 
# #### Use the `latitude_bin0` and `longitude_bin0` to create a `shapely` point for each GEDI shot location. 

# %% [markdown]
# ## 3.1 Subset by Layer and Create a Geodataframe <a id="3.1"></a>

# %% [markdown]
# #### Read in the SDS and take a representative sample (every 100th shot) and append to lists, then use the lists to generate a `pandas` dataframe.

# %%
lonSample, latSample, shotSample, srfSample, degradeSample, beamSample = [], [], [], [], [], []  # Set up lists to store data

# Open the SDS
lats = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][()]
lons = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][()]
shots = gediL1B[f'{beamNames[0]}/shot_number'][()]
srf = gediL1B[f'{beamNames[0]}/stale_return_flag'][()]
degrade = gediL1B[f'{beamNames[0]}/geolocation/degrade'][()]

# Take every 100th shot and append to list
for i in range(len(shots)):
    if i % 1 == 0:
        shotSample.append(str(shots[i]))
        lonSample.append(lons[i])
        latSample.append(lats[i])
        srfSample.append(srf[i])
        degradeSample.append(degrade[i])
        beamSample.append(beamNames[0])
            
# Write all of the sample shots to a dataframe
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 'Latitude': latSample,
                         'Stale Return Flag': srfSample, 'Degrade': degradeSample})
latslons

# %% [markdown]
# #### Above is a dataframe containing columns describing the beam, shot number, lat/lon location, and quality information about each shot.

# %%
# Clean up variables that will no longer be needed
del beamSample, degrade, degradeSample, gediL1B_objs, latSample, lats, lonSample, lons, shotSample, shots, srf, srfSample 

# %% [markdown]
# #### Below, create an additional column called 'geometry' that contains a `shapely` point generated from each lat/lon location from the shot. 

# %%
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
latslons['geometry'] = latslons.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

# %% [markdown]
# #### Next, convert to a `Geopandas` GeoDataFrame.

# %%
# Convert to a Geodataframe
latslons = gp.GeoDataFrame(latslons)
latslons = latslons.drop(columns=['Latitude','Longitude'])
latslons['geometry']

# %% [markdown]
# #### Pull out and plot an example `shapely` point below.

# %%
latslons['geometry'][0]

# %% [markdown]
# ## 3.2 Visualize a GeoDataFrame <a id="3.2"></a>
# #### In this section, use the GeoDataFrame and the `geoviews` python package to spatially visualize the location of the GEDI shots on a basemap and import a geojson file of the spatial region of interest for the use case example: Redwood National Park.

# %%
# Define a function for visualizing GEDI points
def pointVisual(features, vdims):
    return (gvts.EsriImagery * gv.Points(features, vdims=vdims).options(tools=['hover'], height=500, width=900, size=5, 
                                                                        color='yellow', fontsize={'xticks': 10, 'yticks': 10, 
                                                                                                  'xlabel':16, 'ylabel': 16}))

# %% [markdown]
# #### Import a geojson of Redwood National Park as an additional GeoDataFrame. Note that you will need to have downloaded the [geojson](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse/RedwoodNP.geojson) from the bitbucket repo containing this tutorial and have it saved in the same directory as this Jupyter Notebook.

# %%
# redwoodNP = gp.GeoDataFrame.from_file('RedwoodNP.geojson')  # Import geojson as GeoDataFrame
# redwoodNP

# %%
redwoodNP = gp.GeoDataFrame.from_file('/Users/yayayapoop/Library/CloudStorage/OneDrive-TheUniversityofNottingham/Mres - Onedrive/15 04 23 - Sujit Earth engine shape file- WD/Working Folder/17 07 23 - Post Mid review working folder/17 07 23 - New Indices/27 07 23 - GEDI L2 Product most recent 59703/gedi-tutorials-main@d270c58295d/output.geojson')  # Import geojson as GeoDataFrame

# %%
redwoodNP

# %%
redwoodNP['geometry'][0]  # Plot GeoDataFrame

# %% [markdown]
# #### Defining the vdims below will allow you to hover over specific shots and view information about them.

# %%
# Create a list of geodataframe columns to be included as attributes in the output map
vdims = []
for f in latslons:
    if f not in ['geometry']:
        vdims.append(f)
vdims

# %% [markdown]
# #### Below, combine a plot of the Redwood National Park Boundary (combine two `geoviews` plots using `*`) with the point visual mapping function defined above in order to plot (1) the representative GEDI shots, (2) the region of interest, and (3) a basemap layer. 

# %%
# Call the function for plotting the GEDI points
gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None) * pointVisual(latslons, vdims = vdims)

# %%
# Call the function for plotting the GEDI points
gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None) * pointVisual(latslons, vdims = vdims)

# %% [markdown]
# #### Above is a good illustration of the full GEDI orbit (GEDI files are stored as one ISS orbit). One of the benefits of using geoviews is the interactive nature of the output plots. Use the tools to the right of the map above to zoom in and find the shots intersecting Redwood National Park. 
# > (**HINT**: find where the orbit intersects the west coast of the United States)

# %% [markdown]
# #### Below is a screenshot of the region of interest:
# ![alt text](GEDI_L1B_Tutorial_1.png "Sample of GEDI L1B shots in yellow (orbit 2932) plotted over Redwood National Park, USA.")

# %% [markdown]
# #### Side Note: Wondering what the 0's and 1's for `stale_return_flag` and `degrade` mean?

# %%
print(f"stale_return_flag: {gediL1B[b]['stale_return_flag'].attrs['description']}")
print(f"degrade: {gediL1B[b]['geolocation']['degrade'].attrs['description']}")

# %% [markdown]
# #### We will show an example of how to quality filter GEDI data in section 5.
# #### After finding one of the shots within Redwood NP, find the index for that shot number so that we can find the correct waveform to visualize in Section 4. 

# %% [markdown]
# #### Each GEDI shot has a unique shot identifier (shot number) that is available within each data group of the product. The shot number is important to retain in any data subsetting as it will allow the user to link any shot record back to the original orbit data, and to link any shot and its data between the L1 and L2 products. The standard format  for GEDI Shots is as follows:

# %% [markdown]
# ### Shot: 29320619900465601
# > **2932**: Orbit Number    
# **06**: Beam Number  
# **199**: Minor frame number (0-241)   
# **00465601**: Shot number within orbit  

# %%
shot_1 = 167921100300288309
shot_2 = 167921100300288310
shot_3 = 167921100300288311
shot_4 = 167921100300288312


# %%
beamNames

# %%

index1 = np.where(gediL1B[f'{beamNames[0]}/shot_number'][()]==shot_1)[0][0]  # Set the index for the shot identified above
index1

index2 = np.where(gediL1B[f'{beamNames[0]}/shot_number'][()]==shot_2)[0][0]  # Set the index for the shot identified above
index2

index3 = np.where(gediL1B[f'{beamNames[0]}/shot_number'][()]==shot_3)[0][0]  # Set the index for the shot identified above
index3

index4 = np.where(gediL1B[f'{beamNames[0]}/shot_number'][()]==shot_4)[0][0]  # Set the index for the shot identified above
index4

# %%
del latslons  # No longer need the geodataframe used to visualize the full GEDI orbit

# %% [markdown]
# ---
# # 4. Subset and Visualize Waveforms <a id="subsetviswaveforms"></a>
# #### In this section, learn how to extract and subset specific waveforms and plot them using `holoviews`. 

# %% [markdown]
# ## 4.1 Import and Extract Waveforms<a id="4.1"></a>

# %% [markdown]
# #### In order to find and extract the full waveform for the exact index we are interested in, instead of importing the entire waveform dataset (over 1 billion values!), we will use the `rx_sample_count` and `rx_sample_start_index` to identify the location of the waveform that we are interested in visualizing and just extract those values.

# %%
# From the SDS list, use list comprehension to find sample_count, sample_start_index, and rxwaveform
sdsCount = gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_count') and beamNames[0] in g][0]]
sdsStart = gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_start_index') and beamNames[0] in g][0]]
sdsWaveform = [g for g in gediSDS if g.endswith('/rxwaveform') and beamNames[0] in g][0]

# %% [markdown]
# #### Print the description for each of these datasets to better understand how we will use them to extract specific waveforms.

# %%
print(f"rxwaveform is {gediL1B[sdsWaveform].attrs['description']}")

# %% [markdown]
# #### Next, read how to use the rx_sample_count and rx_sample_start_index layers:

# %%
print(f"rx_sample_count is {sdsCount.attrs['description']}")
print(f"rx_sample_start_index is {sdsStart.attrs['description']}")

# %% [markdown]
# #### Use `rx_sample_count` and `rx_sample_start_index` to identify the location of each waveform in `rxwaveform`.

# %%
wfCount = sdsCount[index1]           # Number of samples in the waveform
wfStart = int(sdsStart[index1] - 1)  # Subtract one because python array indexing starts at 0 not 1

# %% [markdown]
# #### Next grab additional information about the shot, including the unique `shot_number`, and lat/lon location.

# %%
index = index1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]

# %% [markdown]
# #### Put everything together to identify the waveform we want to extract:

# %%
print(f"The waveform located at: {str(wfLat)}, {str(wfLon)} (shot ID: {wfShot}, index {index}) is from beam {beamNames[0]} \
      and is stored in rxwaveform beginning at index {wfStart} and ending at index {wfStart + wfCount}")

# %% [markdown]
# #### In order to plot a full waveform, you also need to import the elevation recorded at `bin0` (the first elevation recorded) and `lastbin` or the last elevation recorded for that waveform.

# %%
# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# %% [markdown]
# ### Extract the full waveform using the index start and count:
# #### Below you can see why it is important to extract the specific waveform that you are interested in: almost 1.4 billion values are stored in the rxwaveform dataset!

# %%
print("{:,}".format(gediL1B[sdsWaveform].shape[0]))

# %%
# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

# %% [markdown]
# ## 4.2 Visualize Waveforms
# #### Below, plot the extracted waveform using the elevation difference from bin0 to last_bin on the y axis, and the waveform energy returned or amplitude (digital number = dn) on the x axis. 

# %%
# Find elevation difference from start to finish and divide into equal intervals based on sample_count
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# %%
# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})
wvDF

# %% [markdown]
# #### Next, use the `Holoviews` python package (hv) to plot the waveform.

# %%

# %% [markdown]
# ### Congratulations! You have plotted your first waveform.
# #### Above is a basic line plot showing amplitude (DN) as a function of elevation from the rxwaveform for the specific shot selected. Next, add additional chart elements and make the graph interactive.

# %%
# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
wfVis = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
wfVis

# %% [markdown]
# #### As you can see, the selected shot does not look so interesting--this waveform shape looks more characteristic of some type of low canopy/bare ground than a tree canopy. If you zoom in to the GEDI shot it appears to be over the river itself or possibly a sand bar:
# ![alt text](GEDI_L1B_Tutorial_2.png "GEDI L1B Shot over a river in Redwood National Park, USA.")
# #### Next, plot a couple more waveforms and see if you can capture a shot over the forest.

# %%
latlons = {}                          # Set up a dictionary to hold multiple waveforms
latlons[wfShot] = Point(wfLon,wfLat)  # Create shapely point and add to dictionary

# Retain waveform and quality layers to be exported later
latlonsWF = [waveform]
latlonsEL = [zStretch]
latlonsSRF = [gediL1B[f'{beamNames[0]}/stale_return_flag'][index]]
latlonsD = [gediL1B[f'{beamNames[0]}/geolocation/degrade'][index]]

# %%
# Define which observation to examine by selecting the three indexes prior to the one used above
index = index2

# %%
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

# %%
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

# %%
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

# %%
# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B1 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B1

# %% [markdown]
# #### Now that is starting to look more like a very tall, multi-layered tree canopy! Continue with the next shot:

# %%
# Define which observation to examine by selecting the three indexes prior to the one used above
index = index3

# %%
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

# %%
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

# %%
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

# %%
# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B2 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B2

# %%
# Define which observation to examine by selecting the three indexes prior to the one used above
index = index4

# %%
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

# %%
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

# %%
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

# %%
# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B3 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B3

# %% [markdown]
# #### Lastly, plot the transect of four consecutive waveforms:

# %%
# The "+" symbol will plot multiple saved holoviews plots together
wfVis.opts(width=240)+ visL1B1.opts(width=240) + visL1B2.opts(width=240, labelled=[]) + visL1B3.opts(width=240) 
### wfVis.opts(width=240, labelled=[])

# %% [markdown]
# #### Notice above moving west to east (left to right) along the transect you can see the elevation lowering and the tree canopy decreasing as it encounters the sandbar/river.

# %% [markdown]
# #### Below, use geoviews to plot the location of the four points to verify what is seen above.

# %%
# Convert dict to geodataframe
latlons = gp.GeoDataFrame({'Shot Number': list(latlons.keys()),'rxwaveform Amplitude (DN)': latlonsWF,
                           'rxwaveform Elevation (m)': latlonsEL, 'Stale Return Flag': latlonsSRF, 'Degrade': latlonsD,
                           'geometry': list(latlons.values())})

# %%
latlons['Shot Number'] = latlons['Shot Number'].astype(str)  # Convert shot number from integer to string

# %%
vdims

# %%
# Create a list of geodataframe columns to be included as attributes in the output map
vdims = []
for f in latlons:
    if f not in ['geometry']:
        if 'rxwaveform' not in f:
            vdims.append(f)

# Plot the geodataframe
gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None) * pointVisual(latlons, vdims=vdims)

# %% [markdown]
# #### Above, zoom in on the yellow dots until you can get a clearer view of all four shots.
# #### In the screenshots below, we can match the location of each point with the associated waveform. It appears to confirm the hypothesis that the elevation is decreasing as the shots get closer to the river, and the forest canopy decreases until it is detecting the sand bar/river by the final waveform in the sequence.
# ![alt text](GEDI_L1B_Tutorial_3.png "Plots of Four GEDI L1B Waveforms")
# ![alt text](GEDI_L1B_Tutorial_4.png "Map showing sequence of four GEDI L1B Waveforms in Redwood National Park, USA.")

# %%
del wfVis, visL1B1, visL1B2, visL1B3, waveform, wvDF, zStretch, latlonsWF, latlonsEL, latlonsSRF, latlonsD

# %% [markdown]
# ---
# # 5. Quality Filtering <a id="quality"></a>
# #### Now that you have the desired layers imported as a dataframe for the selected shots, let's perform quality filtering.
# #### Below, remove any shots where the `stale_return_flag` is set to 1 (indicates that a "stale" cue point from the coarse search algorithm is being used) by defining those shots as `nan`. 
# #### The syntax of the line below can be read as: in the dataframe, find the rows "where" the stale return flag is not equal (ne) to 0. If a row (shot) does not meet the condition, set all values equal to `nan` for that row.

# %%
latlons = latlons.where(latlons['Stale Return Flag'].ne(1))  # Set any stale returns to NaN

# %% [markdown]
# #### Below, quality filter even further by using the `degrade` flag (Greater than zero if the shot occurs during a degrade period, zero otherwise).

# %%
latlons = latlons.where(latlons['Degrade'].ne(1))

# %% [markdown]
# #### Below, drop all of the shots that did not pass the quality filtering standards outlined above from the `transectDF`.

# %%
latlons = latlons.dropna()  # Drop all of the rows (shots) that did not pass the quality filtering above

# %%
print(f"Quality filtering complete, {len(latlons)} high quality shots remaining.")

# %% [markdown]
# #### Good news! It looks like all four of the example waveforms passed the initial quality filtering tests. For additional information on quality filtering GEDI data, be sure to check out: https://lpdaac.usgs.gov/resources/faqs/#how-should-i-quality-filter-gedi-l1b-l2b-data.

# %%
del latlons

# %% [markdown]
# ---
# # 6. Plot Profile Transects <a id="plottransects"></a>
# #### In this section, plot a transect subset using waveforms.

# %% [markdown]
# ## 6.1 Subset Beam Transects

# %% [markdown]
# #### Subset down to a smaller transect centered on the waveforms analyzed in the sections above.

# %%
print(index)

# %%
# Grab 50 points before and after the shot visualized above
start = index - 50
end = index + 50 
transectIndex = np.arange(start, end, 1)

# %% [markdown]
# ## 6.2 Plot Waveform Transects

# %% [markdown]
# #### In order to get an idea of the length of the beam transect that you are plotting, you can plot the x-axis as distance, which is calculated below.

# %%
# Calculate along-track distance
distance = np.arange(0.0, len(transectIndex) * 60, 60)  # GEDI Shots are spaced 60 m apart

# %% [markdown]
# #### In order to plot each vertical value for each waveform, you will need to reformat the data structure to match what is needed by `holoviews` Path() capabilities. 

# %%
# Create a list of tuples containing Shot number, and each waveform value at height (z)
wfList = []
for s, i in enumerate(transectIndex):
    if 0 <= i < len(gediL1B['BEAM0110/geolocation/degrade']) and \
       0 <= i < len(gediL1B['BEAM0110/stale_return_flag']) and \
       0 <= i < len(gediL1B['BEAM0110/geolocation/elevation_bin0']) and \
       0 <= i < len(gediL1B['BEAM0110/geolocation/elevation_lastbin']) and \
       0 <= i < len(sdsCount) and \
       0 <= i < len(sdsStart):
       
        # Basic quality filtering from section 5
        if gediL1B['BEAM0110/geolocation/degrade'][i] == 0 and gediL1B['BEAM0110/stale_return_flag'][i] == 0:
            zStart = gediL1B['BEAM0110/geolocation/elevation_bin0'][i]
            zEnd = gediL1B['BEAM0110/geolocation/elevation_lastbin'][i]
            zCount = sdsCount[i]
            zStretch = np.add(zEnd, np.multiply(range(zCount, 0, -1), ((zStart - zEnd) / int(zCount))))
            waveform = gediL1B[sdsWaveform][sdsStart[i]: sdsStart[i] + zCount]
            waves = []
            for z, w in enumerate(waveform):
                waves.append((distance[s], zStretch[z], w))  # Append Distance (x), waveform elevation (y) and waveform amplitude (z)
            wfList.append(waves)
        else:
            print(f"Shot {s} did not pass quality filtering and will be excluded.")
    else:
        print(f"Shot {s} with index {i} is out of range and will be excluded.")


# %% [markdown]
# #### Good news again, it looks like all of the waveforms in our transect passed the quality filtering test.
# 
# ### Below, plot each waveform by using `holoviews` Path() function. This will plot each individual waveform value by distance, with the amplitude plotted in the third dimension in shades of green.

# %%
import warnings
warnings.filterwarnings('ignore')
# Plot the waveform values for the transect with the waveform amplitude being plotted as the 'c' or colormap
l1bVis = hv.Path(wfList, vdims='Amplitude').options(color='Amplitude', clim=(200,405), cmap='Greens', line_width=20, width=950,
                                                    height=500, clabel='Amplitude',colorbar=True, 
                                                    xlabel='Distance Along Transect (m)', ylabel='Elevation (m)', 
                                                    fontsize={'title':16, 'xlabel':16, 'ylabel': 16, 'xticks':12, 'yticks':12, 
                                                              'clabel':12, 'cticks':10}, colorbar_position='right',
                                                    title='GEDI L1B Waveforms across VCS1764: Nov 17, 2021')
l1bVis

# %% [markdown]
# #### If you zoom in to a subset of the transect, you can get even better detail showing the structure of the canopy from the `rxwaveform` dataset, where denser portions of the canopy are seen in darker shades of green.
# ![alt text](GEDI_L1B_Tutorial_5.png "Full waveform amplitude plotted for a transect covering Redwood National Park, USA.")

# %%
del waveform, waves, wfList, zStretch

# %% [markdown]
# ---
# # 7. Spatial Visualization<a id="spatialvisualization"></a>
# #### Section 7 combines many of the techniques learned above including how to import GEDI datasets, perform quality filtering, spatial subsetting, and visualization. 

# %% [markdown]
# ## 7.1 Import, Subset, and Quality Filter All Beams

# %% [markdown]
# #### Below, re-open the GEDI L1B observation--but this time, loop through and import data for all 8 of the GEDI beams.

# %%
beamNames = [g for g in gediL1B.keys() if g.startswith('BEAM')]

# %%
beamNames

# %% [markdown]
# #### Loop through each of the desired datasets (SDS) for each beam, append to lists, and transform into a `pandas` DataFrame.

# %%
# Set up lists to store data
shotNum, dem, zElevation, zHigh, zLat, zLon, srf, degrade, beamI, rxCount, rxStart = ([] for i in range(11))  

# %%
# Loop through each beam and open the SDS needed
for b in beamNames:
    [shotNum.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()]]
    [dem.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/digital_elevation_model') and b in g][0]][()]]
    [zElevation.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/elevation_bin0') and b in g][0]][()]]  
    [zHigh.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/elevation_lastbin') and b in g][0]][()]]  
    [zLat.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/latitude_bin0') and b in g][0]][()]]  
    [zLon.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/longitude_bin0') and b in g][0]][()]]  
    [srf.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/stale_return_flag') and b in g][0]][()]]  
    [degrade.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/degrade') and b in g][0]][()]]  
    [rxCount.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_count') and b in g][0]][()]]  
    [rxStart.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_start_index') and b in g][0]][()]]  
    [beamI.append(h) for h in [b] * len(gediL1B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()])]  

# %% [markdown]
# #### Notice that we did not import the rxwaveform dataset--due to the large size of that dataset, we will wait until after spatial subsetting to import the waveforms.

# %%
# Convert lists to Pandas dataframe
allDF = pd.DataFrame({'Shot Number': shotNum, 'Beam': beamI, 'Latitude': zLat, 'Longitude': zLon, 'Tandem-X DEM': dem,
                      'Elevation bin 0 (m)': zElevation, 'Elevation last bin (m)': zHigh, 'Stale Return Flag': srf,
                      'Degrade Flag': degrade, 'rx Start Index': rxStart, 'rx Sample Count': rxCount})

# %%
del shotNum, dem, zElevation, zHigh, zLat, zLon, srf, degrade, beamI

# %% [markdown]
# ## 7.2 Spatial Subsetting
# #### Below, subset the pandas dataframe using a simple bounding box region of interest. If you are interested in spatially clipping GEDI shots to a geojson region of interest, be sure to check out the GEDI-Subsetter python script available at: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse.

# %%
len(allDF)

# %% [markdown]
# #### Over 3.5 million shots are contained in this single GEDI orbit! Below subset down to only the shots falling within this small bounding box encompassing Redwood National Park. `RedwoodNP` our `geopandas` geodataframe can be called for the "envelope" or smallest bounding box encompassing the entire region of interest. Here, use that as the bounding box for subsetting the GEDI shots.

# %%
redwoodNP.envelope[0].bounds

# %%
minLon, minLat, maxLon, maxLat = redwoodNP.envelope[0].bounds  # Define the min/max lat/lon from the bounds of Redwood NP

# %% [markdown]
# #### Filter by the bounding box, which is done similarly to filtering by quality in section 5 above.

# %%
allDF = allDF.where(allDF['Latitude'] > minLat)
allDF = allDF.where(allDF['Latitude'] < maxLat)
allDF = allDF.where(allDF['Longitude'] > minLon)
allDF = allDF.where(allDF['Longitude'] < maxLon)

# %% [markdown]
# ### NOTE: It may take up to a few minutes to run the cell above.

# %%
allDF = allDF.dropna()  # Drop shots outside of the ROI

# %%
len(allDF)

# %% [markdown]
# #### Notice you have drastically reduced the number of shots you are working with (which will greatly enhance your experience in plotting them below). But first, remove any poor quality shots that exist within the ROI.

# %%
# Set any poor quality returns to NaN
allDF = allDF.where(allDF['Stale Return Flag'].ne(1))
allDF = allDF.where(allDF['Degrade Flag'].ne(1))
allDF = allDF.dropna()
len(allDF)

# %% [markdown]
# #### Next create a `Shapely` Point out of each shot and insert it as the geometry column in the [soon to be geo]dataframe.

# %%
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
allDF['geometry'] = allDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

# %%
# Convert to geodataframe
allDF = gp.GeoDataFrame(allDF)
allDF = allDF.drop(columns=['Latitude','Longitude'])

# %% [markdown]
# ## 7.3 Visualize All Beams: Elevation

# %% [markdown]
# #### Now, using the `pointVisual` function defined in section 3.2, plot the `geopandas` GeoDataFrame using `geoviews`.

# %%
allDF['Shot Number'] = allDF['Shot Number'].astype(str)  # Convert shot number to string

vdims = []
for f in allDF:
    if f not in ['geometry']:
        vdims.append(f)

visual = pointVisual(allDF, vdims = vdims)
visual * gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None)

# %% [markdown]
# #### Feel free to pan and zoom in to the GEDI shots in yellow. 

# %% [markdown]
# ### Now let's not only plot the points in the geodataframe but also add a colormap for Elevation bin0 (m). 

# %%
(gvts.EsriImagery * gv.Points(allDF, vdims=vdims).options(color='Elevation bin 0 (m)',cmap='terrain', size=3, tools=['hover'],
                                                          clim=(min(allDF['Elevation bin 0 (m)']), 
                                                          max(allDF['Elevation bin 0 (m)'])), colorbar=True, clabel='Meters',
                                                          title='GEDI Elevation bin0 over VCS1764: 17 Nov, 2021',
                                                          fontsize={'xticks': 10, 'yticks': 10, 'xlabel':16, 'clabel':12,
                                                                    'cticks':10,'title':16,'ylabel':16})).options(height=500,
                                                                                                                  width=900)

# %%

(gvts.EsriImagery * gv.Points(allDF, vdims=vdims).options(
    color='Elevation bin 0 (m)',
    cmap='terrain',
    size=3,
    tools=['hover'],
    clim=(-5, 10),  # Set the color scale range to -5 to +10
    colorbar=True,
    clabel='Meters',
    title='GEDI Elevation bin0 over VCS1764: 17 Nov, 2021',
    fontsize={'xticks': 10, 'yticks': 10, 'xlabel': 16, 'clabel': 12, 'cticks': 10, 'title': 16, 'ylabel': 16}
)).options(height=500, width=900)

# %% FIg. 10: This Works!!! Dont change it. 
### 1/08/23


import geopandas as gpd
import geoviews as gv
import holoviews as hv
import cartopy.crs as ccrs

# Step 1: Load GeoJSON data
geojson_file_path = '/Users/yayayapoop/Library/CloudStorage/OneDrive-TheUniversityofNottingham/Mres - Onedrive/15 04 23 - Sujit Earth engine shape file- WD/Working Folder/17 07 23 - Post Mid review working folder/17 07 23 - New Indices/27 07 23 - GEDI L2 Product most recent 59703/gedi-tutorials-main@d270c58295d/VCS1764_PAA_UTM46N_2018.geojson'
geojson_data = gpd.read_file(geojson_file_path)

# Assuming you have already defined 'allDF' and 'vdims' variables

# Step 2: Create a GeoViews Path element from the GeoJSON data
geojson_path = gv.Path(geojson_data).options(line_color='red', line_width=2)

# Step 3: Specify the target CRS (Mercator projection)
target_crs = ccrs.Mercator()

# Step 4: Reproject the data to the target CRS
reprojected_data = gv.operation.project(geojson_path, projection=ccrs.PlateCarree(), to=target_crs)

# Step 5: Create the map by overlaying EsriImagery, Points, and reprojected GeoJSON Path
map = (gvts.EsriImagery * gv.Points(allDF, vdims=vdims).options(
    color='Elevation bin 0 (m)',
    cmap='terrain',
    size=3,
    tools=['hover'],
    clim=(-5, 10),
    colorbar=True,
    clabel='Meters',
    title='GEDI Elevation bin0 over VCS1764: 17 Nov, 2021',
    fontsize={'xticks': 10, 'yticks': 10, 'xlabel': 16, 'clabel': 12, 'cticks': 10, 'title': 16, 'ylabel': 16}
) * reprojected_data).options(height=500, width=900)

# Step 6: Load the shapefile and create a GeoViews Path element from it
shapefile_path = '/Users/yayayapoop/Library/CloudStorage/OneDrive-TheUniversityofNottingham/Mres - Onedrive/15 04 23 - Sujit Earth engine shape file- WD/Working Folder/17 07 23 - Post Mid review working folder/17 07 23 - New Indices/27 07 23 - GEDI L2 Product most recent 59703/gedi-tutorials-main@d270c58295d/VCS1764_PAA_UTM46N_2018.shp'  # Replace 'path_to_shapefile' with the actual file path
shapefile_data = gpd.read_file(shapefile_path)
shapefile_path = gv.Path(shapefile_data).options(line_color='blue', line_width=2)

# Step 7: Overlay the shapefile Path on the existing map
map_with_shapefile = map * gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None)

# Step 8: Display the updated map
hv.extension('bokeh')
map_with_shapefile
hv.extension('bokeh')
map_with_shapefile.opts(height=800, width=1200)  # Adjust height and width as desired

# %% Extractions Fig. 78 Extractions of heights within side

import geopandas as gpd
import geoviews as gv
from geoviews import dim

# Assuming 'allDF' is a GeoPandas DataFrame with the point data, and 'redwoodNP' is a GeoPandas DataFrame with the polygon data.

# Step 1: Perform the spatial join to find points inside polygons
points_inside_redwoodNP = gpd.sjoin(allDF, redwoodNP, op='within')

# Step 2: Convert the GeoPandas DataFrame to a GeoViews element
points_inside_redwoodNP_gv = gv.Points(points_inside_redwoodNP, vdims=['Elevation bin 0 (m)'])

# Step 3: Show points on a map, color-coded by elevation
points_inside_redwoodNP_map = points_inside_redwoodNP_gv.opts(
    size=5,
    color='Elevation bin 0 (m)',
    cmap='viridis',  # You can use any other colormap supported by Matplotlib.
    clim=(-3, 3),    # Set the color scale limits to -3 and 3 meters.
    colorbar=True,   # Show the color scale on the map.
    height=800,      # Set the height of the map to 800 pixels.
    width=800       # Set the width of the map to 1200 pixels.
)

# Step 4: Display the map
gv.extension('bokeh')
mm = points_inside_redwoodNP_map * gv.Polygons(redwoodNP['geometry']).opts(line_color='red', color=None)
mm

# %%
import matplotlib.pyplot as plt
column_names = list(points_inside_redwoodNP.columns)

# Step 3: Plot the 4th column against the index
column_to_plot = column_names[3]  # Assuming the 4th column is the one you want to plot
plt.figure(figsize=(8, 6))  # Set the size of the plot
plt.plot(points_inside_redwoodNP.index, points_inside_redwoodNP[column_to_plot], marker='o', linestyle='-', color='b')
plt.xlabel('Index')
plt.ylabel(column_to_plot)
plt.title('Plot of ' + column_to_plot + ' against Index')
plt.grid(True)
plt.show()


# %%
points_inside_redwoodNP

# %%
points_inside_redwoodNP.index

# %%
column_to_plot = column_names[3]  # Assuming the 4th column is the one you want to plot
plt.figure(figsize=(8, 6))  # Set the size of the plot
plt.plot(points_inside_redwoodNP.index, points_inside_redwoodNP["Elevation bin 0 (m)"], marker='o', linestyle='-', color='b')
plt.xlabel('Index')
plt.ylabel(column_to_plot)
plt.title('Plot of ' + column_to_plot + ' against Index')
plt.grid(True)
plt.show()


# %%
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming 'allDF' is a GeoPandas DataFrame with the point data, and 'redwoodNP' is a GeoPandas DataFrame with the polygon data.

# Step 1: Perform the spatial join to find points inside polygons
points_inside_redwoodNP = gpd.sjoin(allDF, redwoodNP, op='within')

# Step 2: Plot the "Elevation bin 0 (m)" column against the index
plt.plot(points_inside_redwoodNP.index, points_inside_redwoodNP["Elevation bin 0 (m)"], marker='o', linestyle='-', color='b')
plt.xlabel('Index')
plt.ylabel('Elevation (m)')
plt.title('Plot of Elevation against Index')
plt.grid(True)
plt.show()
plt.show(block=True)

# %%
ht = points_inside_redwoodNP["Elevation bin 0 (m)"]

plt.plot(ht)
